Ce notebook a été entièrement réalisé par Mélnaie Gomis.

In [1]:
import numpy as np
import glob
import sys
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import gdown
import matplotlib.pyplot as plt

# Chargement des données

In [2]:
# On définit le nom du dossier
dossier_nom = "redshift"
url = "https://drive.google.com/drive/folders/1-tQH6rfB1XoF7ml98yqVn7Z2IQwGMUdK"

# 2. LOGIQUE DE TÉLÉCHARGEMENT
if os.path.exists(dossier_nom):
    print(f"✅ Le dossier '{dossier_nom}' est déjà présent. Pas besoin de retélécharger !")
else:
    print(f"📥 Dossier introuvable. Téléchargement en cours...")
    gdown.download_folder(url, quiet=True, use_cookies=False)
    print("✅ Téléchargement terminé !")

# 3. CRÉATION DE LA LISTE DES FICHIERS (C'est ici que ça plantait avant)
# On scanne le dossier pour créer la liste 'dossier_local' quoi qu'il arrive
pattern = os.path.join(dossier_nom, "*.npz") # ex: redshift/*
dossier_local = glob.glob(pattern)

# 4. AFFICHAGE JOLI
print(f"\nSuccès ! {len(dossier_local)} fichiers trouvés dans '{dossier_nom}'.")
print("Détail des fichiers :")

for chemin in dossier_local:
    # On extrait le nom du dossier et le nom du fichier
    dossier = os.path.dirname(chemin)   # ex: redshift
    fichier = os.path.basename(chemin)  # ex: COSMOS_...npz
    
    # Ton format demandé
    print(f"[{dossier}] :\t {fichier}")

✅ Le dossier 'redshift' est déjà présent. Pas besoin de retélécharger !

Succès ! 8 fichiers trouvés dans 'redshift'.
Détail des fichiers :
[redshift] :	 COSMOS_v11_uijk_0213_photo_UD.npz
[redshift] :	 XMM_LSS_v11_uijk_0162_phot_UD.npz
[redshift] :	 XMM_LSS_v11_uijk_0006_spec_D.npz
[redshift] :	 XMM_LSS_v11_uijk_0177_spec_UD.npz
[redshift] :	 COSMOS_v11_uijk_0020_spec_D.npz
[redshift] :	 COSMOS_v11_uijk_0073_spec_UD.npz
[redshift] :	 COSMOS_v11_uijk_0001_photo_D.npz
[redshift] :	 XMM_LSS_v11_uijk_0162_phot_D.npz


In [3]:
fichiers_npz = glob.glob('redshift/*.npz')
cosmos_files = [f for f in fichiers_npz if "COSMOS" in f]

# Listes séparées pour train et test
train_cubes_list = []
train_infos_list = []
train_flag_list = []

test_cubes_list = []
test_infos_list = []
test_flag_list = []

print(f"Chargement de {len(cosmos_files)} fichiers COSMOS.")

for fichier in cosmos_files:
    try:
        data = np.load(fichier, allow_pickle=True)
        
        # 'info' est un array structuré. 'dtype.names' donne les noms des colonnes.
        info_fields = data['info'].dtype.names
        
        # On vérifie si la *colonne* 'ZSPEC' existe dans ce fichier
        if 'ZSPEC' in info_fields:
            # Ce fichier contient des données de TEST (il a la colonne ZSPEC)
            test_cubes_list.append(data['cube'])
            test_infos_list.append(data['info'])
            test_flag_list.append(data['flag'])
        else:
            # Ce fichier contient des données de TRAIN (pas de colonne ZSPEC)
            train_cubes_list.append(data['cube'])
            train_infos_list.append(data['info'])
            train_flag_list.append(data['flag'])
            
            
    except Exception as e:
        print(f":x: Erreur en chargeant {fichier}: {e}")

print("Chargement terminé.")

Chargement de 4 fichiers COSMOS.
Chargement terminé.


In [4]:
# Création des jeux train/test

# Jeu d’entraînement (basé sur les fichiers SANS ZSPEC)
# On vérifie qu'on a bien trouvé des fichiers de train
if train_infos_list:
    X_train = np.concatenate(train_cubes_list, axis=0)
    infos_train = np.concatenate(train_infos_list, axis=0)
    flags_train = np.concatenate(train_flag_list, axis=0)
    y_train = infos_train['ZPHOT'] # Obtenir ZPHOT depuis les infos de train
    print(f"Objets d'entraînement (ZPHOT) : {len(y_train)}")
    print("X_train :", X_train.shape)
    print("y_train :", y_train.shape)
else:
    print("Aucun fichier d'entraînement (sans ZSPEC) trouvé.")

print("\n---\n")

# Jeu de test (basé sur les fichiers AVEC ZSPEC)
# On vérifie qu'on a bien trouvé des fichiers de test
if test_infos_list:
    X_test = np.concatenate(test_cubes_list, axis=0)
    infos_test = np.concatenate(test_infos_list, axis=0)
    flags_test = np.concatenate(test_flag_list, axis=0)
    y_test = infos_test['ZSPEC'] # Obtenir ZSPEC depuis les infos de test

    # Si on ne garde que les ZSPEC valides
    # mask_zspec_valid = ~np.isnan(y_test) ou ???
    # X_test = X_test[mask_zspec_valid]
    # y_test = y_test[mask_zspec_valid]

    print(f"Objets de test (ZSPEC) : {len(y_test)}")
    print("X_test  :", X_test.shape)
    print("y_test  :", y_test.shape)
else:
    print("Aucun fichier de test (avec ZSPEC) trouvé.")

Objets d'entraînement (ZPHOT) : 12497
X_train : (12497, 64, 64, 9)
y_train : (12497,)

---

Objets de test (ZSPEC) : 27
X_test  : (27, 64, 64, 9)
y_test  : (27,)


# Chargement des codes python

In [ ]:
# 1️⃣ Nom du dossier local
dossier_nom = "redshift/Marie Treyer"

# 2️⃣ ID du dossier Google Drive
folder_id = "1mqIUBkZ3qN82hQ3H5WEdz0YYggMxZe3J"

# 3️⃣ Téléchargement si nécessaire
if os.path.exists(dossier_nom):
    print(f"✅ Le dossier '{dossier_nom}' est déjà présent. Pas besoin de retélécharger !")
else:
    print(f"📥 Dossier introuvable. Téléchargement en cours...")
    try:
        gdown.download_folder(
            id=folder_id,       # utilisation de l'ID
            output=dossier_nom,
            quiet=False,
            use_cookies=False
        )
        print("✅ Téléchargement terminé !")
    except Exception as e:
        print("⚠️ Impossible de télécharger le dossier avec gdown.")
        print("Essayez de le télécharger manuellement depuis le navigateur.")
        raise e

# 4️⃣ Recherche récursive de tous les fichiers .py
pattern = os.path.join(dossier_nom, "**", "*.py")
fichiers_py = glob.glob(pattern, recursive=True)

# 5️⃣ Affichage
print(f"\nSuccès ! {len(fichiers_py)} fichiers .py trouvés dans '{dossier_nom}'.\n")
for chemin in fichiers_py:
    dossier = os.path.dirname(chemin)
    fichier = os.path.basename(chemin)
    print(f"[{dossier}] :\t {fichier}")


✅ Le dossier 'redshift/Marie Treyer' est déjà présent. Pas besoin de retélécharger !

Succès ! 9 fichiers .py trouvés dans 'redshift/Marie Treyer'.

[redshift/Marie Treyer] :	 models_building_blocks.py
[redshift/Marie Treyer] :	 model_multi_modal_simple.py
[redshift/Marie Treyer] :	 __pycache__
[redshift/Marie Treyer] :	 utils
[redshift/Marie Treyer/__pycache__] :	 model_multi_modal_simple.cpython-312.pyc
[redshift/Marie Treyer/__pycache__] :	 models_building_blocks.cpython-312.pyc
[redshift/Marie Treyer/utils] :	 config.py
[redshift/Marie Treyer/utils] :	 __pycache__
[redshift/Marie Treyer/utils/__pycache__] :	 config.cpython-312.pyc


In [6]:
# Chemin absolu du dossier contenant les modules
dossier_nom = "redshift/Marie Treyer"
chemin_absolu = os.path.abspath(dossier_nom)

if chemin_absolu not in sys.path:
    sys.path.append(chemin_absolu)

# Maintenant l'import devrait fonctionner
from model_multi_modal_simple import *

# Création du modèles

In [7]:
import torch
import numpy as np
from model_multi_modal_simple import Model_multi_modal_simple
import utils.config as config

# 1. Préparation des tenseurs
X_train_tensor = torch.tensor(np.transpose(X_train, (0, 3, 1, 2)).copy(), dtype=torch.float32)
X_test_tensor = torch.tensor(np.transpose(X_test, (0, 3, 1, 2)).copy(), dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.copy(), dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test.copy(), dtype=torch.float32).unsqueeze(1)

# 2. Instanciation du modèle
modalities = [list(range(9))]

model = Model_multi_modal_simple(
    in_dim=[64, 64],
    n_outputs=1, 
    modalities=modalities,
    mags_input_size=None, 
    parallel_before_inception_archi=[32, 32],
    parallel_inception_archi=[36, 32, 32, 32, 42, 42], 
    parallel_pooling_before_inceptions=[True, False, False, False, False, False], 
    inception_archi=[109, 101, 101, 101, 156, 156], 
    pooling_before_inceptions=[False, True, False, True, False, False] 
)

device = torch.device(config.CONFIG["DEVICE"])
model.to(device)
model.eval()

# 3. Lancement du Forward Pass (avec le correctif)
print("\n-- Lancement de la prédiction --")
with torch.no_grad(): 
    batch_X = X_test_tensor[:5].to(device)
    
    # CORRECTIF : Création d'un tenseur "ebv" factice (des zéros) de la même taille que le batch
    # Cela permet de contourner le bug d'aplatissement et comble le "+1" attendu par la couche FC
    dummy_ebv = torch.zeros(batch_X.size(0)).to(device)
    
    # On passe notre variable dummy_ebv au modèle
    out_classif, out_regress = model(batch_X, ebv=dummy_ebv)
    
    print(f"Dimension sortie Régression : {out_regress.shape}")
    print(f"Valeurs prédites par le modèle (non entraîné) :\n{out_regress.cpu().numpy()}")

self.in_dim [64, 64]
self.modalities 1

-- Lancement de la prédiction --
Dimension sortie Régression : torch.Size([5, 1])
Valeurs prédites par le modèle (non entraîné) :
[[0.33071488]
 [0.3312552 ]
 [0.33327168]
 [0.33254465]
 [0.33317596]]


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import utils.config as config

# --- CORRECTIF DROPOUT ---
# On désactive le dropout de modalité puisqu'on a regroupé les 9 canaux dans 1 seule modalité
config.CONFIG["MODALITY_DROP_OUT"] = None

# 1. Création du jeu de données et du DataLoader
print("-- Préparation des données d'entraînement --")
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

train_loader = DataLoader(
    train_dataset, 
    batch_size=config.CONFIG["BATCH_SIZE"], 
    shuffle=True, 
    num_workers=config.CONFIG.get("NUM_WORKERS", 0), 
    pin_memory=config.CONFIG.get("PIN_MEMORY", False) 
)

# 2. Définition de la fonction de Perte et de l'Optimiseur
criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3. Paramètres de la boucle d'entraînement
epochs = 10 
device = torch.device(config.CONFIG["DEVICE"]) 

print(f"\n-- Début de l'entraînement sur {device} --")

-- Préparation des données d'entraînement --

-- Début de l'entraînement sur cpu --


In [ ]:
for epoch in range(epochs):
    # Mode entraînement (active les Batch Norms)
    model.train() 
    
    running_loss = 0.0
    
    for batch_idx, (batch_X, batch_y) in enumerate(train_loader):
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        
        # Faux tenseur ebv (zéro) pour ce batch spécifique
        dummy_ebv = torch.zeros(batch_X.size(0)).to(device)
        
        # Étape 1 : Réinitialiser les gradients
        optimizer.zero_grad()
        
        # Étape 2 : Prédiction
        out_classif, out_regress = model(batch_X, ebv=dummy_ebv)
        
        # Étape 3 : Calcul de l'erreur
        loss = criterion(out_regress, batch_y)
        
        # Étape 4 : Rétropropagation
        loss.backward()
        
        # Étape 5 : Mise à jour des poids
        optimizer.step()
        
        # Statistiques
        running_loss += loss.item()
        
        # Affichage régulier
        if (batch_idx + 1) % 1 == 0:
            print(f"Époque [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Perte (MSE): {running_loss/100:.4f}")
            running_loss = 0.0
            
    print(f"==> Fin de l'époque {epoch+1} terminée. <==")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
